In [5]:
from fairseq.models.transformer import TransformerModel
import os
getpwd = os.getcwd()
print(getpwd)
# path_to_your_pretrained_model = '/u/prattisr/phase-2/all_repos/Adversarial_NMT_th/pretrained_models/wmt14.en-fr.joined-dict.transformer'
path_to_your_pretrained_model = getpwd + '/pretrained_models/wmt14.en-fr.joined-dict.transformer'
generator1_pretrained = TransformerModel.from_pretrained(
        path_to_your_pretrained_model,
        checkpoint_file='model.pt',
        bpe='subword_nmt',
        # data_name_or_path='/u/prattisr/phase-2/all_repos/Adversarial_NMT/neural-machine-translation-using-gan-master/data-bin/wmt14_en_fr_raw_sm/50kLines',
        data_name_or_path = getpwd + '/pretrained_models/wmt14.en-fr.joined-dict.transformer',
        bpe_codes = getpwd + '/pretrained_models/wmt14.en-fr.joined-dict.transformer/bpecodes'
)

2024-03-02 08:09:15 | INFO | fairseq.file_utils | Archive name '/root/Adversarial_NMT_th/pretrained_models/wmt14.en-fr.joined-dict.transformer' was not found in archive name list. We assumed '/root/Adversarial_NMT_th/pretrained_models/wmt14.en-fr.joined-dict.transformer' was a path or URL but couldn't find any file associated to this path or URL.
2024-03-02 08:09:15 | INFO | fairseq.file_utils | Archive name '/root/Adversarial_NMT_th/pretrained_models/wmt14.en-fr.joined-dict.transformer' was not found in archive name list. We assumed '/root/Adversarial_NMT_th/pretrained_models/wmt14.en-fr.joined-dict.transformer' was a path or URL but couldn't find any file associated to this path or URL.


TypeError: expected str, bytes or os.PathLike object, not NoneType